In [1]:
using JuMP, Clp

# Multiperiod Planning Problems - ShoeCo example

In [2]:
d = [3000 5000 2000 1000]  # monthly shoe demand

m = Model(Clp.Optimizer)

@variable(m, x[1:4] >= 0 ) # shoes produced in month t=1,2,3,4
@variable(m, w[1:4] >= 0 ) # workers employed in month t=1,2,3,4
@variable(m, o[1:4] >= 0 ) # overtime hours in month t=1,2,3,4
@variable(m, h[1:4] >= 0 ) # workers hired in month t=1,2,3,4
@variable(m, f[1:4] >= 0 ) # workers fired in month t=1,2,3,4
@variable(m, i[1:4] >= 0 ) # shoes in inventory in month t=1,2,3,4

# our objective is to minimize the total cost
@objective(m, Min, 15*sum(x) + 16*sum(o) + 1600*sum(h) + 2000*sum(f)
    + 1500*sum(w) + 3*sum(i))

# constraint on how many total hours are available for shoe production
@constraint(m, production[t in 1:4], 4*x[t] <= 160*w[t] + o[t])

# overtime cannot be greater than 20*# workers in each month
@constraint(m, overtime[t in 1:4], o[t] <= 20*w[t])

# INVENTORY BALANCE #
# amount we start with + amount we produce = amount we sell + amount that carries to next month

# balance inventory in the first month 
@constraint(m, inv_bal_init, 500 + x[1] == d[1] + i[1])
# balance inventory for all months after the first
@constraint(m, inv_bal[t in 2:4], i[t-1] + x[t] == d[t] + i[t])

# WORKER BALANCE #
# number currently employed - number fired + number hired = number workers available

# balance workers in first month
@constraint(m, work_bal_init, 100 - f[1] + h[1] == w[1])
# balance workers in all months after the first
@constraint(m, work_bal[t in 2:4], w[t-1] - f[t] + h[t] == w[t])

# solve this instance of ShoeCo and print relevant solution details
optimize!(m)

# note we use the Array function to turn the list of solution values into an Array.
# this helps with solution legibility.
println("Build ", Array(value.(x')), " shoes each month")
println("Use ", Array(value.(w')), " workers each month")
println("Use ", Array(value.(o')), " overtime hours each month")
println("Cost: ", objective_value(m))

Build [3750.0 3750.0 2000.0 1000.0] shoes each month
Use [93.75000000000001 93.75 50.0 50.00000000000001] workers each month
Use [0.0 0.0 0.0 0.0] overtime hours each month
Cost: 692500.0
Coin0506I Presolve 15 (-1) rows, 23 (-1) columns and 44 (-2) elements
Clp0006I 0  Obj 37500 Primal inf 6746.7591 (5)
Clp0006I 13  Obj 692500
Clp0000I Optimal - objective value 692500
Coin0511I After Postsolve, objective 692500, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 692500 - 13 iterations time 0.002, Presolve 0.00


## Post solution analysis:

The problem with this solution is that the value of the number of workers each month is not an integer. It should be an integer. Maybe use a floor function from math to optimize it and only get integer values?

# Han's stressful Flight

A minimax problem where you have to minimize all the maximums. Use the epigraph trick here where you define the max function as t and then minimize t

In [3]:
m = Model(Clp.Optimizer)

@variable(m, x >= 0) # variable representing the speed Han flies
@variable(m, t) # epigraph trick variable

# minimize the epigraph variable t
@objective(m, Min, t)

@constraint(m, t >= -2*x + 20) # t must be >= Leia's stress function
@constraint(m, t >= -(1/10)*x + 10) # t must be >= Chewbacca's stress function
@constraint(m, t >= x + 1.75) # t must be >= C3P0's stress function
@constraint(m, x >= 6) # Han has to fly fast enough to outrun the Empire
@constraint(m, x <= 12) # Han can't exceed the Falcon's maximum speed

# solve this instance of the minimax problem
optimize!(m)

# to help make the output neater, we can round the solution values to 2 decimal places
# also note that we use each person's stress function to calculate their stress at the final speed
println("Han should fly at ", round(value(x),digits=2), " parsecs/min")
println("Leia's stress: ", -2*round(value(x),digits=2) + 20)
println("Chewbacca's stress: ", -(1/10)*value(x) + 10)
println("C-3P0's stress: ", round(value(x),digits=2) + 1.75)

# the value of t tells us what the minimized maximum stress level is
println("Minimax stress level: ", round(value(t),digits=2))

Han should fly at 7.5 parsecs/min
Leia's stress: 5.0
Chewbacca's stress: 9.25
C-3P0's stress: 9.25
Minimax stress level: 9.25
Coin0506I Presolve 3 (-2) rows, 2 (0) columns and 6 (-2) elements
Clp0006I 0  Obj 8.7 Primal inf 2.2135934 (1)
Clp0006I 2  Obj 9.25
Clp0000I Optimal - objective value 9.25
Coin0511I After Postsolve, objective 9.25, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 9.25 - 2 iterations time 0.002, Presolve 0.00
